互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息
机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：

    contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
    is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
    is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
    contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
    contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
    contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
    contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
    is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
    with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
    contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
    contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
    contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
    allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
    contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
    contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
    has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
    traffic：该网站的流量大小。1表示大，0表示小；
    google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；
    
    y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
    原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。

将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [3]:
# 导入必要的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

data = pd.read_csv('fraudulent.csv')

# 计算每列的缺失值比例
missing_ratio = data.isnull().mean()

# 设定一个阈值，比如缺失比例超过30%的列将被删除
threshold = 0.3
columns_to_drop = missing_ratio[missing_ratio > threshold].index

# 删除缺失值过多的列
data = data.drop(columns=columns_to_drop)

# 对剩余的缺失值使用众数进行填充
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# 3. 分离特征和标签
X = data_imputed.drop('y', axis=1)
y = data_imputed['y']

# 4. 划分训练集和测试集（随机种子设置为1）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y
)

print("训练集大小:", X_train.shape)
print("测试集大小:", X_test.shape)

# 5. 特征缩放（对于某些模型如SVM和KNN可能需要）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6. 选择和训练模型
# 我们将尝试几种不同的分类器并比较它们的F1分数

# 定义模型
models = {
    'Logistic Regression': LogisticRegression(random_state=1, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=1),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(random_state=1, probability=True)
}

# 训练和评估每个模型
for name, model in models.items():
    # 对于需要缩放的模型，使用缩放后的数据
    if name in ['K-Nearest Neighbors', 'Support Vector Machine', 'Logistic Regression']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    print(f"{name} 的F1分数: {f1:.4f}")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

# 7. 选择最佳模型（假设F1分数最高的模型为最佳）
# 这里以逻辑回归为例，假设它表现最好

# 示例：使用逻辑回归进行最终模型训练和评估
best_model = LogisticRegression(random_state=1, max_iter=1000)
best_model.fit(X_train_scaled, y_train)
y_pred_best = best_model.predict(X_test_scaled)
best_f1 = f1_score(y_test, y_pred_best)
print(f"最佳模型（Logistic Regression）的F1分数: {best_f1:.4f}")
print("分类报告:")
print(classification_report(y_test, y_pred_best))


训练集大小: (8068, 17)
测试集大小: (2018, 17)
Logistic Regression 的F1分数: 0.8537
              precision    recall  f1-score   support

         0.0       0.93      0.94      0.93      1382
         1.0       0.87      0.84      0.85       636

    accuracy                           0.91      2018
   macro avg       0.90      0.89      0.89      2018
weighted avg       0.91      0.91      0.91      2018

--------------------------------------------------
Decision Tree 的F1分数: 0.8710
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94      1382
         1.0       0.90      0.84      0.87       636

    accuracy                           0.92      2018
   macro avg       0.91      0.90      0.91      2018
weighted avg       0.92      0.92      0.92      2018

--------------------------------------------------
K-Nearest Neighbors 的F1分数: 0.8440
              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93      1382
   

output：   
训练集大小: (8068, 17)   
测试集大小: (2018, 17)   
          Logistic Regression 的F1分数: 0.8537
                    precision    recall  f1-score   support

               0.0       0.93      0.94      0.93      1382
               1.0       0.87      0.84      0.85       636

          accuracy                           0.91      2018
          macro avg       0.90      0.89      0.89      2018
          weighted avg       0.91      0.91      0.91      2018

          --------------------------------------------------
          Decision Tree 的F1分数: 0.8710
                    precision    recall  f1-score   support

               0.0       0.93      0.96      0.94      1382
               1.0       0.90      0.84      0.87       636

          accuracy                           0.92      2018
          macro avg       0.91      0.90      0.91      2018
          weighted avg       0.92      0.92      0.92      2018

          --------------------------------------------------
          K-Nearest Neighbors 的F1分数: 0.8440
                    precision    recall  f1-score   support

               0.0       0.92      0.94      0.93      1382
               1.0       0.87      0.82      0.84       636

          accuracy                           0.90      2018
          macro avg       0.89      0.88      0.89      2018
          weighted avg       0.90      0.90      0.90      2018

          --------------------------------------------------
          Support Vector Machine 的F1分数: 0.8698
                    precision    recall  f1-score   support

               0.0       0.93      0.95      0.94      1382
               1.0       0.90      0.85      0.87       636

          accuracy                           0.92      2018
          macro avg       0.91      0.90      0.91      2018
          weighted avg       0.92      0.92      0.92      2018

          --------------------------------------------------
          最佳模型（Logistic Regression）的F1分数: 0.8537
          分类报告:
                    precision    recall  f1-score   support

               0.0       0.93      0.94      0.93      1382
               1.0       0.87      0.84      0.85       636

          accuracy                           0.91      2018
          macro avg       0.90      0.89      0.89      2018
          weighted avg       0.91      0.91      0.91      2018